<h1 style="text-align: center;">AAG - Méthodes Classiques en IA</h1>


## Naive Bayes

### Préparation du dataset & création des corpus

**<ins>Dataset R6 :</ins>**
<style>
    .heatMap {
        width: 70%;
        text-align: center;
    }
    .heatMap th {
        background: grey;
        word-wrap: break-word;
        text-align: center;
    }

</style>

<ins>Types de features</ins>
<div class="heatMap">
									

|   Nom des features           |  Type       | description |
|:---------------:|:-----------:|:-----------:|
|   skillrank  |   Nom (string)   | rang du joueur |
|   operator  |   Nom (string)  | nom du personnage |
|   primaryweapon   |   Nom (string)   | nom de l'arme principale |
|   secondaryweapon   |   nom (string)   | nom de l'arme secondaire |
|   secondarygadget   |   nom (string)   | nom du gadget utilisé |
|   k/d   |   numeric (int)   | ratio mort/kill (1 si >1 sinon 0) |
|   target   |   numeric (int)   | ratio victoire/partie (1 si >50% sinon 0)
</div>

<ins>Séparation du corpus</ins>
<div class="heatMap">

|                 | Train (2/3) | Test(1/3) | Total / type de w/r |
|:---------------:|:-----:|:----:|:------:|
| 1 (w/r positif)  |   398  |  199  |   597   |
| 0 (w/r negatif) |   1035  |  518  |   1553   |
| Total / corpus |   1433 | 717  |  2150   |

</div>

on peut utiliser le naive bayes car nos descripteurs sont tous discrétisé / discrets

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Chargement du dataset dans une DataFrame
dataset_R6 = pd.read_csv("./DatasetR6/S5_operators.csv", sep=';')

# on calcule le w/r
dataset_R6.insert(dataset_R6.shape[1],"w/r", (dataset_R6["nbwins"]/dataset_R6["nbpicks"]), True) 
dataset_R6.drop(columns=["nbwins","nbpicks","role"], inplace=True)

# on calcule le k/d
dataset_R6.loc[dataset_R6['nbdeaths'] == 0] = 1
# on a une ligne qui fait que des 1 partout 
dataset_R6.drop(axis=0, index=dataset_R6[dataset_R6['operator'] == 1].index, inplace=True)
dataset_R6['k/d'] = dataset_R6['nbkills'] / dataset_R6['nbdeaths']


# on merge et enleve les colonnes plateformes et dateid sans perdre les données associées
dataset_R6 = dataset_R6.groupby(['skillrank','operator', 'primaryweapon', 'secondaryweapon', 'secondarygadget'], as_index=False)[['w/r', 'k/d']].mean()

# discretisation de w/r
dataset_R6.insert(dataset_R6.shape[1],"target", (dataset_R6["w/r"]>=0.5).astype(int), True) 

dataset_R6.drop(columns=["w/r"], inplace=True)

#discrétisation de k/d
dataset_R6['k/d'] = (dataset_R6["k/d"]>=1.0).astype(int)


# changement de la taille d'affichage des dataframes
#pd.set_option('display.max_rows', dataset_R6.shape[0]+1)


# création des corpus annotés et non annotés
corpus_annot = dataset_R6.copy()
corpus_annot = corpus_annot[['target']]

corpus_non_annot = dataset_R6.copy()
corpus_non_annot.drop(columns=["target"], inplace=True)

x_train, x_test, y_train, y_test = train_test_split(corpus_non_annot, corpus_annot, test_size=1/3, random_state=42, stratify=corpus_annot)

corpus_test_non_annot = x_test.copy()

corpus_train = x_train.copy()
corpus_train['target'] = y_train.copy()

corpus_verif = x_test.copy()
corpus_verif['target'] = y_test.copy()

corpus_verif.head(10)


,skillrank,operator,primaryweapon,secondaryweapon,secondarygadget,k/d,target
583,Copper,SWAT-PULSE,M1014,M45 MEUSOC,NITRO CELL,0,0
962,Gold,GIGN-RESERVE,F2,LFP586,FRAG GRENADE,0,0
992,Gold,GIGN-ROOK,P90,P9,DEPLOYABLE SHIELD,0,1
1612,Silver,GIGN-ROOK,SG-CQB,P9,DEPLOYABLE SHIELD,0,1
368,Copper,GIGN-RESERVE,SG-CQB,LFP586,DEPLOYABLE SHIELD,0,0
211,Bronze,SPETSNAZ-FUZE,AK-12,PMM,STUN GRENADE,1,0
1199,Gold,SWAT-RESERVE,556XI,5.7 USG,STUN GRENADE,0,0
2122,Unranked,SWAT-RESERVE,556XI,5.7 USG,FRAG GRENADE,0,0
1997,Unranked,SAS-RESERVE,M590A1,P226 MK 25,FRAG GRENADE,0,0
43,Bronze,GIGN-MONTAGNE,Shield,P9,STUN GRENADE,0,0


In [11]:
dic_vraisemblance = {}

for nom_attribut in corpus_test_non_annot.columns:
    dic_vraisemblance[nom_attribut] = corpus_test_non_annot[nom_attribut].unique()

if('target' in corpus_test_non_annot.columns):
    corpus_test_non_annot.drop(columns=["target"], inplace=True)

# Recherche du descripteur qui minimise le taux d'erreur
for nom_attribut in corpus_test_non_annot.columns:
    attribut = corpus_test_non_annot[nom_attribut]
    tab_val = []
    dic_evidence = {}
    for valeur in attribut.unique():
        dic_wr = {0:0, 1:0}
        #print(f"pour la valeur {valeur} la target max est \n {corp_train.loc[corp_train[nom_attribut] == valeur]['target'].value_counts().idxmax()}")
        dic_evidence[valeur] = corpus_train.loc[corpus_train[nom_attribut] == valeur]['target'].value_counts().idxmax()
        
    dic_vraisemblance[nom_attribut] = dic_evidence.copy()
    
dic_vraisemblance

{'skillrank': {'Copper': 0,
  'Gold': 0,
  'Silver': 0,
  'Bronze': 0,
  'Unranked': 0,
  'Diamond': 0,
  'Platinum': 0},
 'operator': {'SWAT-PULSE': 0,
  'GIGN-RESERVE': 0,
  'GIGN-ROOK': 1,
  'SPETSNAZ-FUZE': 0,
  'SWAT-RESERVE': 0,
  'SAS-RESERVE': 0,
  'GIGN-MONTAGNE': 0,
  'SAS-MUTE': 1,
  'SPETSNAZ-RESERVE': 0,
  'SPETSNAZ-GLAZ': 0,
  'SPETSNAZ-TACHANKA': 0,
  'SWAT-CASTLE': 0,
  'SWAT-ASH': 1,
  'SAS-SMOKE': 0,
  'GSG9-IQ': 0,
  'GIGN-DOC': 0,
  'SAT-ECHO': 0,
  'G.E.O.-JACKAL': 0,
  'SPETSNAZ-KAPKAN': 0,
  'SWAT-THERMITE': 0,
  'SAS-THATCHER': 0,
  'GSG9-RESERVE': 0,
  'GIGN-TWITCH': 0,
  'G.E.O.-MIRA': 0,
  'JTF2-BUCK': 0,
  'NAVYSEAL-BLACKBEARD': 0,
  'GSG9-BANDIT': 1,
  'SAS-SLEDGE': 0,
  'NAVYSEAL-VALKYRIE': 1,
  'JTF2-FROST': 1,
  'SAT-HIBANA': 0,
  'GSG9-BLITZ': 0,
  'GSG9-JAGER': 0,
  'BOPE-CAPITAO': 0,
  'BOPE-CAVEIRA': 1},
 'primaryweapon': {'M1014': 0,
  'F2': 0,
  'P90': 0,
  'SG-CQB': 0,
  'AK-12': 0,
  '556XI': 0,
  'M590A1': 0,
  'Shield': 0,
  'MP5K': 0,
  '6P41'